In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the dataset
df = pd.read_csv("SCOA_A4.csv")

# Identify feature and target columns
X = df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
y_categorical = df['species']

# Encode the target column
le = LabelEncoder()
y = le.fit_transform(y_categorical)

print("Dataset loaded successfully.")
print(f"Features (X) shape: {X.shape}")
print(f"Target (y) shape: {y.shape}")
print("First 5 rows of features (X):\n", X.head())
print("First 5 encoded target values (y):\n", y[:5])
print("Original species labels (first 5):\n", y_categorical[:5])
print("Encoded target classes:", le.classes_)

Dataset loaded successfully.
Features (X) shape: (150, 4)
Target (y) shape: (150,)
First 5 rows of features (X):
    sepal_length  sepal_width  petal_length  petal_width
0           5.1          3.5           1.4          0.2
1           4.9          3.0           1.4          0.2
2           4.7          3.2           1.3          0.2
3           4.6          3.1           1.5          0.2
4           5.0          3.6           1.4          0.2
First 5 encoded target values (y):
 [0 0 0 0 0]
Original species labels (first 5):
 0    setosa
1    setosa
2    setosa
3    setosa
4    setosa
Name: species, dtype: object
Encoded target classes: ['setosa' 'versicolor' 'virginica']


In [3]:
import numpy as np
import random
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

# --- Genetic Algorithm Setup ---
POP_SIZE = 20      # number of individuals
N_GENERATIONS = 10 # iterations
MUTATION_RATE = 0.2

# Chromosome: [max_depth, min_samples_split]
def create_chromosome():
    return [random.randint(1, 20), random.randint(2, 10)]

def fitness(chromosome):
    max_depth, min_samples_split = chromosome
    model = DecisionTreeClassifier(max_depth=max_depth,
                                   min_samples_split=min_samples_split,
                                   random_state=42) # Added random_state for reproducibility
    scores = cross_val_score(model, X, y, cv=5)
    return scores.mean()

def selection(population, fitnesses):
    # Select the best two individuals for crossover
    idx = np.argsort(fitnesses)[-2:]
    return [population[idx[0]], population[idx[1]]]

def crossover(parent1, parent2):
    # Perform single-point crossover
    point = random.randint(0, len(parent1)-1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def mutate(chromosome):
    # Apply mutation to max_depth
    if random.random() < MUTATION_RATE:
        chromosome[0] = random.randint(1, 20)
    # Apply mutation to min_samples_split
    if random.random() < MUTATION_RATE:
        chromosome[1] = random.randint(2, 10)
    return chromosome

# --- Run GA ---
population = [create_chromosome() for _ in range(POP_SIZE)]

print("Starting Genetic Algorithm...")
for gen in range(N_GENERATIONS):
    fitnesses = [fitness(chromo) for chromo in population]
    print(f"Generation {gen+1}/{N_GENERATIONS} - Best Fitness: {max(fitnesses):.4f}")

    new_population = []
    parents = selection(population, fitnesses)

    # Generate new population through crossover and mutation
    for _ in range(POP_SIZE // 2):
        child1, child2 = crossover(parents[0], parents[1])
        new_population.append(mutate(child1))
        new_population.append(mutate(child2))

    population = new_population

# Best result from the final population
fitnesses = [fitness(chromo) for chromo in population]
best_idx = np.argmax(fitnesses)
best_hyperparameters = population[best_idx]
best_accuracy = fitnesses[best_idx]

print("\nGenetic Algorithm Finished.")
print(f"Best Hyperparameters: max_depth={best_hyperparameters[0]}, min_samples_split={best_hyperparameters[1]}")
print(f"Best Accuracy: {best_accuracy:.4f}")

Starting Genetic Algorithm...
Generation 1/10 - Best Fitness: 0.9733
Generation 2/10 - Best Fitness: 0.9733
Generation 3/10 - Best Fitness: 0.9733
Generation 4/10 - Best Fitness: 0.9733
Generation 5/10 - Best Fitness: 0.9733
Generation 6/10 - Best Fitness: 0.9733
Generation 7/10 - Best Fitness: 0.9733
Generation 8/10 - Best Fitness: 0.9733
Generation 9/10 - Best Fitness: 0.9733
Generation 10/10 - Best Fitness: 0.9733

Genetic Algorithm Finished.
Best Hyperparameters: max_depth=3, min_samples_split=7
Best Accuracy: 0.9733


In [ ]:
"""
Below is a clear and viva-ready explanation covering:

* Meaning & significance of the code
* Implementation steps
* Interpretation of output
* Key viva questions with short answers

---

## 🎯 **Objective of the Code**

The goal of this code is to **automatically find the best hyperparameters** (`max_depth`, `min_samples_split`) for a **Decision Tree Classifier** using a **Genetic Algorithm (GA)** — instead of manually tuning them.

* **Dataset Used:** Iris dataset (`SCOA_A4.csv`)
* **Algorithm Used:** Genetic Algorithm (GA)
* **Model Tuned:** Decision Tree Classifier
* **Evaluation Metric:** Mean Accuracy (via 5-fold Cross-Validation)

---

## ⚙️ **Step-by-Step Explanation**

### **1️⃣ Data Loading and Encoding**

```python
df = pd.read_csv("SCOA_A4.csv")
X = df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
y_categorical = df['species']
```

* **Features (X):** Numerical measurements of Iris flowers.
* **Target (y):** Flower species — categorical labels: `setosa`, `versicolor`, `virginica`.

Encoding:

```python
le = LabelEncoder()
y = le.fit_transform(y_categorical)
```

* Converts species names into numeric form:
  `setosa → 0`, `versicolor → 1`, `virginica → 2`.

✅ **Purpose:** Machine learning models require numerical target values.

---

### **2️⃣ GA Setup**

```python
POP_SIZE = 20      # number of individuals (candidate solutions)
N_GENERATIONS = 10 # number of iterations
MUTATION_RATE = 0.2
```

Each individual (chromosome) represents a set of hyperparameters:

```python
[max_depth, min_samples_split]
```

* `max_depth`: how deep the tree can grow
* `min_samples_split`: minimum samples required to split a node

Both affect **model complexity and accuracy**.

---

### **3️⃣ Creating Initial Population**

```python
def create_chromosome():
    return [random.randint(1, 20), random.randint(2, 10)]
```

* Generates random hyperparameter values for the first generation.

---

### **4️⃣ Fitness Function**

```python
def fitness(chromosome):
    max_depth, min_samples_split = chromosome
    model = DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_samples_split, random_state=42)
    scores = cross_val_score(model, X, y, cv=5)
    return scores.mean()
```

* Each individual is evaluated by training a Decision Tree using its parameters.
* The **fitness** is the **average cross-validation accuracy**.
* Higher fitness = better model performance.

✅ **Goal:** Maximize accuracy → find the chromosome with best fitness.

---

### **5️⃣ Selection**

```python
def selection(population, fitnesses):
    idx = np.argsort(fitnesses)[-2:]
    return [population[idx[0]], population[idx[1]]]
```

* Selects **two best chromosomes** (parents) with highest fitness.
* These will produce the next generation through crossover.

---

### **6️⃣ Crossover**

```python
def crossover(parent1, parent2):
    point = random.randint(0, len(parent1)-1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
```

* Combines parts of two parents to form two new children.
* Mimics **biological reproduction**, encouraging good traits to mix.

Example:
Parent1 = [10, 5], Parent2 = [15, 3] → Crossover → Child1 = [10, 3], Child2 = [15, 5].

---

### **7️⃣ Mutation**

```python
def mutate(chromosome):
    if random.random() < MUTATION_RATE:
        chromosome[0] = random.randint(1, 20)
    if random.random() < MUTATION_RATE:
        chromosome[1] = random.randint(2, 10)
    return chromosome
```

* Randomly alters genes (hyperparameters) with some probability.
* Keeps diversity in population → avoids local optima.

---

### **8️⃣ Evolution Loop**

```python
for gen in range(N_GENERATIONS):
    fitnesses = [fitness(chromo) for chromo in population]
    print(f"Generation {gen+1}/{N_GENERATIONS} - Best Fitness: {max(fitnesses):.4f}")
```

At each generation:

1. Evaluate fitness of all individuals.
2. Select top parents.
3. Create new generation via crossover & mutation.
4. Replace old population with new one.

This process repeats for 10 generations.

---

## 📊 **Output Interpretation**

### **Part 1: Dataset Info**

```
Features (X) shape: (150, 4)
Target (y) shape: (150,)
Encoded target classes: ['setosa' 'versicolor' 'virginica']
```

* Confirms that dataset has 150 samples, 4 features, and 3 classes.

---

### **Part 2: GA Progress**

```
Generation 1/10 - Best Fitness: 0.9733
...
Generation 10/10 - Best Fitness: 0.9733
```

* Best accuracy (fitness) stabilizes early, meaning optimal hyperparameters are found quickly.

---

### **Part 3: Final Results**

```
Best Hyperparameters: max_depth=3, min_samples_split=7
Best Accuracy: 0.9733
```

✅ **Interpretation:**

* Decision Tree with **depth = 3** and **min_samples_split = 7** achieves **97.33% accuracy** (cross-validated).
* Indicates an excellent fit — model is accurate and not overfitting.

---

## 🧩 **Significance of GA in this Implementation**

| Aspect                         | Explanation                                                                  |
| ------------------------------ | ---------------------------------------------------------------------------- |
| **Purpose**                    | Automate hyperparameter tuning                                               |
| **Approach**                   | Use population-based stochastic optimization                                 |
| **Why GA?**                    | It explores a wide search space and can escape local minima                  |
| **Advantage over Grid Search** | Fewer evaluations, faster convergence, and flexible parameter representation |
| **Result**                     | Finds near-optimal Decision Tree parameters yielding high accuracy           |

---

## 🗣️ **Viva Quick Answers**

**Q1. What is being optimized in this code?**
→ The hyperparameters `max_depth` and `min_samples_split` of a Decision Tree.

**Q2. What does the fitness function represent?**
→ The 5-fold cross-validation accuracy of the Decision Tree.

**Q3. What are the roles of crossover and mutation?**
→ Crossover mixes good traits from two parents; mutation introduces randomness to maintain diversity.

**Q4. Why use Genetic Algorithm for tuning?**
→ Because GA performs global search and can find optimal parameters efficiently without exhaustive search.

**Q5. What is the stopping criterion here?**
→ The GA stops after a fixed number of generations (`N_GENERATIONS = 10`).

**Q6. What is the final model’s performance?**
→ ~97.33% accuracy with optimal parameters (max_depth=3, min_samples_split=7).

---

## 🧮 **Summary Table**

| Step | Description                                            |
| ---- | ------------------------------------------------------ |
| 1️⃣  | Load and encode the Iris dataset                       |
| 2️⃣  | Define Decision Tree hyperparameters as chromosomes    |
| 3️⃣  | Evaluate fitness via 5-fold accuracy                   |
| 4️⃣  | Select best parents based on fitness                   |
| 5️⃣  | Apply crossover and mutation to create next generation |
| 6️⃣  | Repeat for 10 generations                              |
| 7️⃣  | Output best hyperparameters and accuracy               |

---

"""

In [ ]:
"""
---

## 🧠 1. Conceptual Overview

**Genetic Algorithm (GA)** is a **stochastic optimization** technique inspired by **Charles Darwin’s theory of natural evolution**.
It mimics how biological organisms evolve over generations through **selection**, **crossover**, and **mutation**.

GA is widely used for optimization problems where:

* The **search space is large or complex** (non-linear, multi-modal, non-differentiable).
* **Gradient-based methods** (like in deep learning) are not applicable.

---

## 🧬 2. Core Biological Analogy

| Biological Process | GA Equivalent                | Meaning                                    |
| ------------------ | ---------------------------- | ------------------------------------------ |
| Population         | Set of candidate solutions   | Each candidate = one possible solution     |
| Chromosome         | Encoded form of a solution   | E.g., [max_depth, min_samples_split]       |
| Gene               | One part of a chromosome     | E.g., max_depth = 5                        |
| Fitness            | Quality measure              | How good the solution is                   |
| Selection          | Choosing fittest individuals | For reproduction                           |
| Crossover          | Combining parent genes       | Produces new offspring (solutions)         |
| Mutation           | Random change in genes       | Introduces diversity to avoid local minima |

---

## 🧩 3. Step-by-Step Working Mechanism

Let’s link the GA steps to your code and theory.

### **Step 1 — Initialization**

Generate an initial random population of `POP_SIZE` chromosomes.

[
P^{(0)} = { x_1, x_2, \dots, x_N }
]

Each chromosome encodes a possible set of hyperparameters.

In your case:

```python
def create_chromosome():
    return [random.randint(1, 20), random.randint(2, 10)]
```

👉 Each chromosome = `[max_depth, min_samples_split]`.

---

### **Step 2 — Fitness Evaluation**

Each chromosome’s **fitness** measures how well it solves the problem.

[
f(x_i) = \text{performance of the model with parameters } x_i
]

In your case:

```python
scores = cross_val_score(model, X, y, cv=5)
return scores.mean()
```

→ Fitness = **average accuracy** from 5-fold cross-validation.

Mathematically:
[
f(x_i) = \frac{1}{k} \sum_{j=1}^{k} \text{Accuracy}_j
]

---

### **Step 3 — Selection**

Choose the **best-performing individuals** (highest fitness) to reproduce.

Your code:

```python
idx = np.argsort(fitnesses)[-2:]
return [population[idx[0]], population[idx[1]]]
```

Mathematically, this simulates **“survival of the fittest”**:
[
P_{\text{selected}} = \text{argmax}_{x_i \in P^{(t)}} f(x_i)
]

Other popular selection methods:

* **Roulette Wheel Selection:** Probability proportional to fitness
  ( P(x_i) = \frac{f(x_i)}{\sum_j f(x_j)} )
* **Tournament Selection:** Pick the best among a random group.

---

### **Step 4 — Crossover (Recombination)**

Parents combine their genes to form new offspring.

Your code:

```python
point = random.randint(0, len(parent1)-1)
child1 = parent1[:point] + parent2[point:]
child2 = parent2[:point] + parent1[point:]
```

Mathematically, for two parents ( x_1, x_2 ):
[
\text{child}_1 = (x_1[:c], x_2[c:])
]
[
\text{child}_2 = (x_2[:c], x_1[c:])
]
where ( c ) = crossover point.

This creates **genetic diversity** by blending two good solutions.

---

### **Step 5 — Mutation**

Randomly change some genes to maintain exploration and avoid premature convergence.

Your code:

```python
if random.random() < MUTATION_RATE:
    chromosome[0] = random.randint(1, 20)
```

Mathematically, for each gene ( g ):
[
g' =
\begin{cases}
\text{random value}, & \text{if } r < p_m \
g, & \text{otherwise}
\end{cases}
]

where ( p_m ) is the mutation probability (e.g., 0.2).

Mutation ensures the algorithm **doesn’t get stuck** in a local optimum.

---

### **Step 6 — Replacement**

Replace the old population with the newly created offspring:
[
P^{(t+1)} = \text{offspring from } P^{(t)}
]

This iterative process continues for `N_GENERATIONS`.

---

## 🧮 4. Mathematical Foundation (Optimization View)

The GA can be viewed as a **stochastic optimization process** over the search space ( S ), trying to maximize a fitness function ( f: S \rightarrow \mathbb{R} ).

### Objective:

[
\max_{x \in S} f(x)
]

GA performs **randomized search** guided by fitness proportionate selection.

The population evolution can be expressed as a **Markov process**:
[
P^{(t+1)} = M(P^{(t)})
]
where ( M ) is a stochastic transformation determined by selection, crossover, and mutation.

Over iterations:
[
\lim_{t \to \infty} \mathbb{E}[f(P^{(t)})] \to f^*
]
(meaning the expected best fitness approaches the global optimum).

---

## 🧩 5. Genetic Algorithm Pseudocode (Mathematical Form)

```
1. Initialize P(0) = {x1, x2, ..., xN} randomly
2. Evaluate f(xi) for all xi ∈ P(0)
3. Repeat until termination condition:
      a. Select parents from P(t)
      b. Perform crossover to produce offspring
      c. Apply mutation to offspring
      d. Form P(t+1) from offspring
      e. Evaluate fitness f(xi)
4. Return the best individual x* with max f(x)
```

---

## 📊 6. Visualization (Simplified Example)

Let’s say we are optimizing:

[
f(x) = \sin(x) + \cos(2x)
]

GA steps:

* Randomly initialize ( x \in [0, 10] )
* Evaluate ( f(x) )
* Keep top solutions → Crossover → Mutate → Repeat
  → Over generations, the population “moves” toward the region where ( f(x) ) is maximum.

---

## 🔬 7. Advantages of GA

✅ Works well with **nonlinear**, **discontinuous**, or **non-differentiable** functions.
✅ Doesn’t require gradient information.
✅ Can escape local minima.
✅ Easy to parallelize (evaluate multiple solutions at once).

---

## ⚠️ 8. Limitations

❌ Computationally expensive (evaluating each chromosome’s fitness repeatedly).
❌ Convergence can be slow.
❌ Sensitive to parameter tuning (population size, mutation rate).
❌ May overfit or oscillate if mutation/crossover rates are not balanced.

---

## 🧩 9. Example in Your Code Context

Your **fitness function** used **Decision Tree accuracy** as a measure of “fitness.”

So the GA is **searching the best combination** of:

* `max_depth`
* `min_samples_split`

that **maximizes**:
[
f(\text{chromosome}) = \text{Average Cross-Validation Accuracy}
]

After 10 generations, it found:

```
Best Hyperparameters: max_depth=3, min_samples_split=7
Best Accuracy: 0.9733
```

That’s your **fittest individual** — the “best evolved species.”

---

## 🧠 10. Summary Table

| Concept     | Symbol / Example    | Meaning                                   |
| ----------- | ------------------- | ----------------------------------------- |
| Population  | ( P^{(t)} )         | All candidate solutions at generation *t* |
| Chromosome  | [3, 7]              | Decision Tree hyperparameters             |
| Gene        | 3 or 7              | A single value in chromosome              |
| Fitness     | ( f(x_i) )          | Accuracy of the model                     |
| Selection   | argmax(fitness)     | Pick top individuals                      |
| Crossover   | combine(p1, p2)     | Mix parent genes                          |
| Mutation    | random_change(gene) | Introduce diversity                       |
| Termination | after N generations | Stop evolution                            |
| Output      | Best fitness        | Optimal solution                          |

---

"""